In [1]:
from general_trader import GeneralTrader
import polars as pl



In [10]:
import polars as pl
#from polars import DataFrame, Expr
# Make sure ValidationError is imported here
from pydantic import BaseModel, field_validator, ValidationError, ConfigDict
from typing import Set, Any

In [2]:

# Create a sample valid DataFrame
valid_data = {
    'ticker': ['AAPL', 'AAPL', 'MSFT', 'MSFT'],
    'date': ['2023-01-01', '2023-01-02', '2023-01-01', '2023-01-02'],
    'open': [170.0, 171.0, 280.0, 281.0],
    'high': [172.0, 173.0, 282.0, 283.0],
    'low': [169.0, 170.0, 279.0, 280.0],
    'close': [171.5, 172.5, 281.5, 282.5],
    'volume': [100000, 110000, 80000, 85000]
}
df_valid = pl.DataFrame(valid_data).with_columns(
    pl.col('date').str.strptime(pl.Date, "%Y-%m-%d") # Convert date string to Date type
)

# Create a sample invalid DataFrame (missing 'volume')
invalid_data_missing_col = {
    'ticker': ['AAPL'], 'date': ['2023-01-01'], 'open': [170.0],
    'high': [172.0], 'low': [169.0], 'close': [171.5]
    # 'volume' is missing
}
df_invalid_missing = pl.DataFrame(invalid_data_missing_col).with_columns(
        pl.col('date').str.strptime(pl.Date, "%Y-%m-%d")
)

    # Create a sample invalid DataFrame (duplicate primary key)
    # Note: Primary key check is commented out in validator by default for performance
    # invalid_data_duplicate_pk = {
    #     'ticker': ['AAPL', 'AAPL'], # Duplicate ticker/date
    #     'date': ['2023-01-01', '2023-01-01'],
    #     'open': [170.0, 171.0], 'high': [172.0, 173.0], 'low': [169.0, 170.0],
    #     'close': [171.5, 172.5], 'volume': [100000, 110000]
    # }
    # df_invalid_duplicate = pl.DataFrame(invalid_data_duplicate_pk).with_columns(
    #      pl.col('date').str.strptime(pl.Date, "%Y-%m-%d")
    # )


    # Define a sample Polars expression
    #my_filter = (pl.col("close") > 100) & (pl.col("volume") > 50000)
my_filter = None
    # --- Test Cases ---

print("--- Testing Valid Instantiation ---")
try:
    trader = GeneralTrader(df_ohlc=df_valid, initialFilter=my_filter)
    print("Successfully created GeneralTrader instance.")
    print("DataFrame Schema:", trader.df_ohlc.schema)
    print("Initial Filter:", trader.initialFilter)
except ValidationError as e:
    print("Validation Error (unexpected):", e)




--- Testing Valid Instantiation ---
Successfully created GeneralTrader instance.
DataFrame Schema: Schema([('ticker', String), ('date', Date), ('open', Float64), ('high', Float64), ('low', Float64), ('close', Float64), ('volume', Int64)])
Initial Filter: None


In [5]:
trader.df_ohlc

ticker,date,open,high,low,close,volume,sum_A,mean_B,A_plus_B
str,date,f64,f64,f64,f64,i64,f64,f64,f64
"""AAPL""",2023-01-01,170.0,172.0,169.0,171.5,100000,910.0,224.5,341.5
"""AAPL""",2023-01-02,171.0,173.0,170.0,172.5,110000,910.0,224.5,343.5
"""MSFT""",2023-01-01,280.0,282.0,279.0,281.5,80000,910.0,224.5,561.5
"""MSFT""",2023-01-02,281.0,283.0,280.0,282.5,85000,910.0,224.5,563.5


In [4]:
indicators = {
    "sum_A": pl.col("high").sum(),
    "mean_B": pl.col("low").mean(),
    #"bad_stuff": 5,
    "A_plus_B": pl.col("open") + pl.col("close"),
}
trader.add_columns(indicators)

GeneralTrader(df_ohlc=shape: (4, 10)
┌────────┬────────────┬───────┬───────┬───┬────────┬───────┬────────┬──────────┐
│ ticker ┆ date       ┆ open  ┆ high  ┆ … ┆ volume ┆ sum_A ┆ mean_B ┆ A_plus_B │
│ ---    ┆ ---        ┆ ---   ┆ ---   ┆   ┆ ---    ┆ ---   ┆ ---    ┆ ---      │
│ str    ┆ date       ┆ f64   ┆ f64   ┆   ┆ i64    ┆ f64   ┆ f64    ┆ f64      │
╞════════╪════════════╪═══════╪═══════╪═══╪════════╪═══════╪════════╪══════════╡
│ AAPL   ┆ 2023-01-01 ┆ 170.0 ┆ 172.0 ┆ … ┆ 100000 ┆ 910.0 ┆ 224.5  ┆ 341.5    │
│ AAPL   ┆ 2023-01-02 ┆ 171.0 ┆ 173.0 ┆ … ┆ 110000 ┆ 910.0 ┆ 224.5  ┆ 343.5    │
│ MSFT   ┆ 2023-01-01 ┆ 280.0 ┆ 282.0 ┆ … ┆ 80000  ┆ 910.0 ┆ 224.5  ┆ 561.5    │
│ MSFT   ┆ 2023-01-02 ┆ 281.0 ┆ 283.0 ┆ … ┆ 85000  ┆ 910.0 ┆ 224.5  ┆ 563.5    │
└────────┴────────────┴───────┴───────┴───┴────────┴───────┴────────┴──────────┘, initialFilter=None)

In [22]:
df_newStuff

ticker,date,open,high,low,close,volume,sum_A,mean_B,A_plus_B
str,date,f64,f64,f64,f64,i64,f64,f64,f64
"""AAPL""",2023-01-01,170.0,172.0,169.0,171.5,100000,910.0,224.5,341.5
"""AAPL""",2023-01-02,171.0,173.0,170.0,172.5,110000,910.0,224.5,343.5
"""MSFT""",2023-01-01,280.0,282.0,279.0,281.5,80000,910.0,224.5,561.5
"""MSFT""",2023-01-02,281.0,283.0,280.0,282.5,85000,910.0,224.5,563.5
